In [1]:
'''
Import libraries
'''
import math
import numpy as np
import matplotlib.pyplot as plt

from qiskit import *
from qiskit.visualization import plot_histogram

from qiskit.providers.fake_provider import FakeAuckland
from qiskit.providers.fake_provider import FakeGuadalupeV2

# Importing standard Qiskit libraries
from qiskit.providers.ibmq import least_busy
from qiskit.circuit.random import random_circuit

from qiskit.primitives import Sampler as prim_Sampler
from qiskit_aer.primitives import Sampler as aer_Sampler

from qiskit_ibm_runtime import Session, Options
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Sampler as run_Sampler

from qiskit.quantum_info import Statevector
from qiskit.circuit.library.standard_gates import RZGate

import warnings
warnings.filterwarnings("ignore")

from qiskit.circuit import Qubit, QuantumRegister, CircuitInstruction, Instruction
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager            

from qiskit.providers.aer.noise import NoiseModel

In [2]:
'''
Qiskit Service backends
'''
#Use this for old account.
#service = QiskitRuntimeService()

#Use this for NSF account.
service = QiskitRuntimeService(name="SR-NSF")
service.backends()

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_kyoto')>,
 <IBMBackend('ibm_osaka')>,
 <IBMBackend('ibmq_qasm_simulator')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('simulator_statevector')>]

In [20]:
def createW(n):
    '''
    Creates n-qubit W state.
    '''
    temp = QuantumCircuit(n)
    temp.ry(2*np.arccos(1/np.sqrt(n)), 0)
    for i in range(n-2):
        temp.ch(i, i+1)
    for i in range(n-1):
        temp.cx(n-i-2, n-i-1)
    temp.x(0)
    return temp

In [3]:
def get_dist(state):
    '''
    Get true distribution of Hamming weight outcomes for the input state.
    '''
    backend = Aer.get_backend('statevector_simulator')
    result = execute(state, backend).result().get_statevector()
    dic = {}
    for i in range(len(np.asarray(result))):
        if np.abs(result[i])>=1e-6:
            st = bin(i)
            count = 0
            for k in range(len(st)):
                if (st[k] == '1'):
                    count += 1
            if (count in dic):
                dic[count] += np.real((result[i])*np.conj(result[i]))
            else:
                dic[count] = np.real((result[i])*np.conj(result[i]))

    return dic

In [13]:
'''
Helper functions to postprocess input for encoded IQFT
'''

def getSumfromString(string):
    lis = [int(i) for i in list(string)]
    return sum(lis)%2

def checkList(a, b):
    for i in range(len(a)):
        print(a[i] - b[i])

def getDecimal(lisOfbinDigits):
    return int(''.join(map(str, lisOfbinDigits)), 2)

def getNewIndex(k, s, i):
    bin_i = bin(i)[2:].zfill(k*s)
    outlist = []
    
    for l in range(k):
        outlist.append(getSumfromString(bin_i[l*s:(l+1)*s]))
    
    return getDecimal(outlist[::-1])

def postProcessDist(dist, k, s):
    newDist = {}
    for i in range(2**k):
        newDist[i] = 0.0
    
    for i in dist:
        newDist[getNewIndex(k, s, i)] += dist[i]

    for i in range(2**k):
        if (newDist[i] == 0.0):
            del newDist[i]
    return dict(sorted(newDist.items()))

def getListofValidString(s):
    lisValStr = []
    for i in range(2**s):
        bin_i = bin(i)[2:].zfill(s)
        if getSumfromString(bin_i) == 1:
            lisValStr.append(i)
    return lisValStr

In [14]:
def create_SC_RepIQFTCircuit(k, s):
    '''
    Create encoded IQFT circuit. Uses c_if to create classically controlled rotations.
    '''
    lisValStr = getListofValidString(s)
    
    qc = QuantumCircuit(k*s)    
    for i in range(k):
        qc.add_register(ClassicalRegister(size=s))

    for i in range(k):
        for j in range(i):
            for item in lisValStr:
                qc.rz(-np.pi/(2**(i-j)), i*s).c_if(qc.cregs[j], item)
        qc.barrier()
        qc.h(range(i*s, i*s + s))
        qc.measure(range(i*s, (i+1)*s), range(i*s, (i+1)*s))
        qc.barrier()
    
    return qc

In [15]:
def createRepIQFTCircuit(k, s):
    '''
    Create encoded IQFT circuit and apply on input state.
    Still uses train of CNOTS to get each bit. Only used for testing.
    '''

    qc = QuantumCircuit(k*s, k*s)    
    for i in range(k):
        for j in range(i):
            qc.crz(np.pi/(2**(i-j)), (j+1)*s-1, i*s)
        qc.h(range(i*s, i*s + s))
    
        for l in range(s-1):
            qc.cx(i*s+l, i*s+l+1)
    
    for i in range(k):
        for j in range(s-1):
            qc.cx(i*s-j-2, i*s-j-1)
    
    qc.barrier()
    qc.measure(list(range(k*s)), list(range(k*s)))
    return qc

In [16]:
def repetitionEncoding(s):
    '''
    Creates s-qubit repetition code encoder of depth log(s).
    '''
    repEnc = QuantumCircuit(s)
    if (s==1):
        return repEnc
    if (s==2):
        repEnc.cx(0, 1)
        return repEnc
    
    x = int(np.floor(s/2))
    repEnc.cx(0, x)
    repEnc = repEnc.compose(repetitionEncoding(x), range(0, x))
    repEnc = repEnc.compose(repetitionEncoding(s-x), range(x, s))
    return repEnc

In [9]:
def createQuantumCircuit(state, s):
    '''
    Creates quantum circuit for the problem. 
    '''
    n = state.num_qubits
    k = int(np.ceil(np.log2(n+1)))
    N = 2**k - 1
    
    circ = QuantumCircuit(k*s + n)
    #Create Quantum Circuit here
    
    circ.h([i*s for i in range(k)])
    circ.append(state, list(range(k*s, k*s + n)))
    
    for i in range(k):
        y = 2**i
        circ.append(RZGate(np.pi*n*y/(N+1)), [s*i])
    circ.barrier()
    
    for i in range(k):
        circ = circ.compose(repetitionEncoding(s), range(i*s, i*s + s))
    circ.barrier()
    
    startQubit = k*s
    
   
    for i in range(k):
        t = (k-i-1)*s
        y = 2**(k-i-1)
        for j in range(n):
            tqubit = j
            circ.append(RZGate(2*np.pi*y/(N+1)).control(1), [t + j%s, tqubit+startQubit])
        circ.barrier()
        
    circ = circ.compose(create_SC_RepIQFTCircuit(k, s).reverse_bits(), range(k*s), range(k*s))
    return circ

In [10]:
s = 2
n = 2
k = int(np.ceil(np.log2(n+1)))

state = random_circuit(n, 5)

encCirc = createQuantumCircuit(state, s)

In [11]:
true_dist = get_dist(state)
print(true_dist)

{0: 0.8190074382541596, 1: 0.09049628087292047, 2: 0.09049628087292044}


In [12]:
sampler = aer_Sampler()
repDist = postProcessDist(sampler.run(encCirc, shots=1e4).result().quasi_dists[0], k, s)
print(repDist)

{0: 0.8180000000000001, 1: 0.0936, 2: 0.0884}


In [17]:
def measOutcomes(qCirc, shots, choice = 0):
    '''
    Calculates the expectation value for the different backends.
    '''
    if choice == 0: 
        sampler = aer_Sampler()
        job = sampler.run(qCirc, shots = shots)
        
    elif choice == 1:
        fake_backend = FakeGuadalupeV2()
        noise_model = NoiseModel.from_backend(fake_backend)
        options = Options()
        options.simulator = {
            "noise_model": noise_model,
            "coupling_map": fake_backend.coupling_map,
            "seed_simulator": 42
        }
        options.optimization_level = 3
        options.resilience_level = 1
        
        sampler = run_Sampler(session=session1, options=options)
        job = sampler.run(qCirc, shots=shots)

    return job

In [15]:
LB = least_busy(service.backends(filters=lambda x: x.configuration().n_qubits >= 20 and not x.configuration().simulator))
print(LB)

session1 = Session(service=service, backend="ibmq_qasm_simulator")
session2 = Session(service=service, backend=LB.name)

<IBMBackend('ibm_osaka')>


In [16]:
def writeToFile(filename, qasm, s, lis_exp):
    '''
    Write the data to a text file to be used to plot the data. The format is as follows:
        1. QASM description of state preparation unitary
        2. true_dist
        3. dic_noiseless
        4. dic_shot_noise
        5. dic_noisy_sim
        6. dic_quantum
    '''
    file = open(filename, "w+")
    file.write(qasm)
    file.write(str(s)+"\n")
    for i in range(len(lis_exp)):
        file.write(str(lis_exp[i]) + "\n")
    file.close()

In [18]:
pm1 = generate_preset_pass_manager(optimization_level=2, target=service.backend('ibmq_qasm_simulator').target)

In [18]:
def populate_dicts(data_experiments, list_experiments):
    '''
    Extracts output data from IBM cloud jobs.
    '''
    for i in range(len(data_experiments)):
        if (str(type(data_experiments[i][2])) == "<class 'qiskit_ibm_runtime.runtime_job.RuntimeJob'>"):
            if str(data_experiments[i][2].status()) == "JobStatus.DONE":
                data_experiments[i][2] = postProcessDist(data_experiments[i][2].result().quasi_dists[0], int(np.ceil(np.log2(list_experiments[i][0].num_qubits+1))), list_experiments[i][1])     
    return data_experiments

In [19]:
def get_data_experiments(list_experiments, shots):
    '''
    Runs all experiments and returns data array.
    '''
    data_experiments = []
    for i in list_experiments:
        data_experiments.append([])
    
    for i in range(len(list_experiments)):
        state = list_experiments[i][0]
        s = list_experiments[i][1]
        
        true_dist = get_dist(state)
        dic_shot_noise = postProcessDist(measOutcomes(createQuantumCircuit(state, s), shots, 0).result().quasi_dists[0], int(np.ceil(np.log2(state.num_qubits+1))), s)
        dic_noisy = measOutcomes(pm1.run(createQuantumCircuit(state, s)), shots, 1)
        
        #dic_noisy_sim and dic_quantum hold a JOB now. Not the distribution.
        data_experiments[i] = [true_dist, dic_shot_noise, dic_noisy]
    
    return data_experiments

In [23]:
"Define a list of quantum states to test Hamming weight. Need to do it this way to reduce queueing time."
list_experiments = []
list_names = ["3-qubit-GHZ-SC-s-1", "3-qubit-GHZ-SC-s-2", "3-qubit-GHZ-SC-s-3"]

#1
state = QuantumCircuit(3)
state.h(0)
state.cx(0, 1)
state.cx(1, 2)
s = 1
list_experiments.append((state, s))

#2
state = QuantumCircuit(3)
state.h(0)
state.cx(0, 1)
state.cx(1, 2)
s = 2
list_experiments.append((state, s))

#3
state = QuantumCircuit(3)
state.h(0)
state.cx(0, 1)
state.cx(1, 2)
s = 3
list_experiments.append((state, s))

In [24]:
data_experiments = get_data_experiments(list_experiments, 10000)

In [26]:
data_experiments = populate_dicts(data_experiments, list_experiments)
print(data_experiments)

[[{0: 0.5000000000000001, 3: 0.4999999999999999}, {0: 0.4991, 3: 0.5009}, {0: 0.4343275841463477, 1: 0.0578007356492167, 2: 0.06756564599554048, 3: 0.440306034208895}], [{0: 0.5000000000000001, 3: 0.4999999999999999}, {0: 0.4989, 3: 0.5011}, {0: 0.42986030563617117, 1: 0.08832833190785645, 2: 0.07247604749935202, 3: 0.4093353149566203}], [{0: 0.5000000000000001, 3: 0.4999999999999999}, {0: 0.5035, 3: 0.49649999999999994}, {0: 0.40751811049458814, 1: 0.10034614804270361, 2: 0.11308383266337876, 3: 0.37905190879932954}]]


In [27]:
for i in range(len(list_experiments)):
    writeToFile("Data Files/"+ list_names[i] +".txt", list_experiments[i][0].qasm(), list_experiments[i][1], data_experiments[i])